In [1]:
#@title Installation
from IPython.display import clear_output
import torch
import codecs


git_repo_rot13 = 'uggcf://tvguho.pbz/snprshfvba/snprshfvba'
git_repo = codecs.decode(git_repo_rot13, 'rot 13')

#@markdown Choose the branch of  to install: 'master' is the stable latest release, 'next' is the latest beta for in-development features.
git_branch ="master" #@param ["master", "next"]


if torch.cuda.is_available() and git_branch == "master":
  !apt-get update
  !apt-get install -y nvidia-cuda-toolkit
  device="cuda"
  print("Using GPU")
elif torch.cuda.is_available() and git_branch == "next":
  # remove cuda 12.2
  !apt-get --purge remove "*cublas*" "cuda*" "nsight*" -y
  !apt autoremove -y
  !apt-get autoclean -y
  # download cuda 12.4
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
  !sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
  !wget https://developer.download.nvidia.com/compute/cuda/12.4.0/local_installers/cuda-repo-ubuntu2204-12-4-local_12.4.0-550.54.14-1_amd64.deb
  !sudo dpkg -i cuda-repo-ubuntu2204-12-4-local_12.4.0-550.54.14-1_amd64.deb
  !sudo cp /var/cuda-repo-ubuntu2204-12-4-local/cuda-*-keyring.gpg /usr/share/keyrings/
  !sudo apt-get update
  !sudo apt-get -y install cuda-toolkit-12-4
  device="cuda"
  print("Using GPU")
else:
  device="cpu"
  print("Using CPU")


!git clone $git_repo --branch $git_branch --single-branch

directory_rot13 = 'snprshfvba'
directory = codecs.decode(directory_rot13, 'rot 13')

%cd /content/$directory

if device=="cuda" and git_branch == "master":
  !python install.py --onnxruntime cuda-12.2 --skip-conda
elif device=="cuda" and git_branch == "next":
  !python install.py --onnxruntime cuda-12.4 --skip-conda
elif device == "cpu":
  !python install.py --onnxruntime default --skip-conda

clear_output()
print("Installed!")

Installed!


In [ ]:
#@title Run UI

import codecs
import re

%cd /content/$directory

#@markdown The type of tunnel you wanna use for seeing the public link, so if the API of one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudfare", "LocalTunnel"]

#@markdown Also when using Ngrok, Cloudfare or LocalTunnel as the Tunnel, you need to wait for the Local URL to appear, and only after that click on the  Public URL above it.

#@markdown Use the option under this only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "Show_1_steps_colab.ipynb" #@param {type:"string"}


if Tunnel == "Gradio":
  file_path_ui_rot13 = "/pbagrag/snprshfvba/snprshfvba/hvf/ynlbhgf/qrsnhyg.cl"
  file_path_ui = codecs.decode(file_path_ui_rot13, 'rot 13')
  !sed -i 's/show_api = False/show_api=False,share=True/g' $file_path_ui
elif Tunnel == "Ngrok":
  !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  clear_output()
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
  !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
  # download cloudfare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudfare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')


if device=="cuda" and git_branch == "master":
  !python run.py --execution-providers cuda
elif device == "cpu" and git_branch == "master":
  !python run.py --execution-providers cpu
elif device=="cuda" and git_branch == "next":
  !python $directory\.py run --execution-providers cuda
elif device=="cpu" and git_branch == "next":
  !python $directory\.py run --execution-providers cpu